In [ ]:
### District Summary
# Create a high-level snapshot, in a DataFrame, of the district's key metrics, including the following:
# * Total schools
# * Total students
# * Total budget
# * Average math score
# * Average reading score
# * % passing math (the percentage of students who passed math)
# * % passing reading (the percentage of students who passed reading)
# * % overall passing (the percentage of students who passed math AND reading